In [1]:
import pickle 
with open('df_ner_analysis.pkl','rb') as f:
    df = pickle.load(f)

### Can you give us an everday situation where a firm publicly supports a value, similar to the sellers that endorsed values in this study?

In [2]:
ORG, PER, LOC, MISC = [], [], [], []

for row in df['ner_output']:
        if row != "[]":
            for entity in row:
                for key, value in entity.items():
                    if key == 'ORG':
                        ORG.append(value)
                    elif key == 'PER':
                        PER.append(value)
                    elif key == 'LOC':
                        LOC.append(value)
                    elif key == 'MISC':
                        MISC.append(value)

In [3]:
#'Admit not knowing / remembering' = '[]'
#'Concrete example' = 'ORG'
#'Generic example' = 'MISC'
#'No example or non-firm example' = '[]' or 'PER' or 'MISC'
#'Nonsense' = '[]'

In [4]:
df['ner_output'][16]

[{'ORG': 'Twitter'},
 {'ORG': 'Microsoft'},
 {'ORG': 'Sony'},
 {'MISC': 'LGBTQ'},
 {'MISC': 'Pride'},
 {'ORG': 'Twitter'},
 {'MISC': 'Pride'},
 {'MISC': 'LGBTQ'}]

Human encodings are prone to biases:

Black Lives Matter supports racial inequities. classified as Nonsense although a lot of racial movements are classified as Generic Example

Samme as with "Mostly LGBTQ people and Gun Control people are mostly aligned."

In [5]:
for row in df['ner_output']:
    if row != "[]":
        for entity in row:
            for key, value in entity.items():
                print(key)

ORG
ORG
MISC
ORG
PER
ORG
ORG
ORG
ORG
ORG
ORG
MISC
MISC
MISC
MISC
ORG
MISC
MISC
ORG
ORG
ORG
MISC
MISC
ORG
MISC
MISC
ORG
ORG
MISC
ORG
ORG
ORG
MISC
ORG
ORG
ORG
MISC
ORG
ORG
MISC
ORG
ORG
MISC
ORG
ORG
ORG
ORG
MISC
ORG
ORG
ORG
MISC
ORG
ORG
MISC
ORG
MISC
MISC
MISC
ORG
MISC
ORG
ORG
ORG
MISC
MISC
ORG
ORG
ORG
ORG
ORG
ORG
ORG
ORG
ORG
ORG
ORG
MISC
PER
MISC
ORG
ORG
MISC
ORG
ORG
ORG
ORG
ORG
ORG
MISC
ORG
ORG
ORG
MISC
ORG
MISC
ORG
LOC
LOC
ORG
ORG
MISC
ORG
ORG
ORG
ORG
ORG
MISC
ORG
LOC
ORG
ORG
ORG
ORG
ORG
ORG
LOC
ORG
MISC
MISC
ORG
ORG
ORG
MISC
PER
ORG
ORG
ORG
ORG
ORG
ORG
ORG
ORG
MISC
ORG
ORG
ORG
ORG
MISC
ORG
MISC
PER
LOC
ORG
MISC
ORG
ORG
ORG
MISC
ORG
ORG
ORG
MISC
ORG
ORG
ORG
ORG
MISC
ORG
ORG
MISC
MISC
MISC
ORG
MISC
ORG
ORG
ORG
ORG
ORG
ORG
MISC
MISC
MISC
ORG
MISC
ORG
ORG
ORG
MISC
ORG
MISC
MISC
ORG
ORG
MISC
ORG
PER
ORG
ORG
ORG
MISC
LOC
MISC
ORG
MISC
MISC
MISC
ORG
ORG
MISC
MISC
PER
ORG
ORG
MISC
ORG
ORG
LOC
ORG
MISC
MISC
MISC
MISC
MISC
PER
ORG
ORG
ORG
ORG
ORG
ORG
MISC
MISC
ORG
MISC
MISC
MISC
ORG
ORG
ORG
MIS

In [ ]:
from collections import OrderedDict

def dedupe_entities(entity_list):
    seen = set()
    unique = []
    for d in entity_list:
        if not isinstance(d, dict) or not d:
            continue
        key, val = next(iter(d.items()))
        if (key, val) not in seen:
            seen.add((key, val))
            unique.append({key: val})

    return unique

df['ner_output_unique'] = df['ner_output'].apply(dedupe_entities)

In [7]:
df['ner_output_unique'][16]

[{'ORG': 'Twitter'},
 {'ORG': 'Microsoft'},
 {'ORG': 'Sony'},
 {'MISC': 'LGBTQ'},
 {'MISC': 'Pride'}]

In [8]:
df['prediction_binary'] = 0

In [9]:
#'Admit not knowing / remembering' = '[]'
#'Concrete example' if contains key 'ORG'
#'Generic example' if contains key 'MISC'
#'No example or non-firm example' if '[]' or contains 'PER' or 'MISC'
#'Nonsense' = '[]'

In [ ]:
df['prediction_binary'] = 0

for idx, row in df.iterrows():
    ner_list = row['ner_output']
    keys = {k for d in ner_list for k in d}
    empty = len(ner_list) == 0 

    label = row['rv_label']

    if label == "Admit not knowing / remembering" and empty:
        df.at[idx, 'prediction_binary'] = 1

    elif label == "Concrete example" and 'ORG' in keys:
        df.at[idx, 'prediction_binary'] = 1

    elif label == "Generic example" and 'MISC' in keys:
        df.at[idx, 'prediction_binary'] = 1

    elif label == "No example or non-firm example" and (empty or keys & {'ORG', 'PER', 'MISC'}):
        df.at[idx, 'prediction_binary'] = 1

    elif label == "Nonsense" and empty:
        df.at[idx, 'prediction_binary'] = 1


In [12]:
true_labels = df['rv_label'].value_counts()
true_labels = true_labels.to_frame()
true_labels.rename(columns={'count':'binary_true'}, inplace=True)

In [13]:
df_eval = df[['rv_label', 'prediction_binary']]

In [14]:
df

,recognition_value,rv_label,ner_output,ner_output_unique,prediction_binary
0,"No, I cannot think of a firm that explicitly e...",Admit not knowing / remembering,[],[],1
1,McDonalds publicly supports families with sick...,Concrete example,"[{'ORG': 'McDonalds'}, {'ORG': 'RonaldMcDonald...","[{'ORG': 'McDonalds'}, {'ORG': 'RonaldMcDonald...",1
2,I cannot think of one but there are firms that...,Admit not knowing / remembering,[{'MISC': 'LGBTQ'}],[{'MISC': 'LGBTQ'}],0
3,Sackcloth makes blankets and works to be clima...,Concrete example,[],[],0
4,Bud Light and Dylan Mulvaney but they found ou...,Concrete example,"[{'ORG': 'BudLight'}, {'PER': 'DylanMulvaney'}...","[{'ORG': 'BudLight'}, {'PER': 'DylanMulvaney'}...",1
...,...,...,...,...,...
795,Victorias Secret supports LGBTQ,Concrete example,"[{'ORG': 'VictoriasSecret'}, {'MISC': 'LGBTQ'}]","[{'ORG': 'VictoriasSecret'}, {'MISC': 'LGBTQ'}]",1
796,Companies often align themselves with values r...,Generic example,[],[],0
797,The company BLQK puts 25% of their profit back...,Concrete example,[{'ORG': 'BLQK'}],[{'ORG': 'BLQK'}],1
798,Black Lives Matter supports racial inequities.,Nonsense,[{'MISC': 'BlackLivesMatter'}],[{'MISC': 'BlackLivesMatter'}],0


In [15]:
df_predicted = df_eval.groupby(['rv_label']).sum()
df_eval = df_predicted.join(true_labels, on='rv_label')

In [16]:
df_eval['perc_covered'] = (df_eval['prediction_binary'] / df_eval['binary_true'] * 100).round(1)

In [17]:
df_eval

,prediction_binary,binary_true,perc_covered
rv_label,,,
Admit not knowing / remembering,80,82,97.6
Concrete example,332,474,70.0
Generic example,29,175,16.6
No example or non-firm example,49,50,98.0
Nonsense,17,19,89.5


In [18]:
print(df_eval["perc_covered"].mean())

74.34


Misleading categories: 'No example or non-firm example' as this classifies basically anything. 

Wrong Ground truth labels: 
"Nonsense" and "No example or non-firm example" classifying organizations. 

Concrete examples weirdly are not identified although the exact same tokens do appear in identified cases.
Generic example	is not the purpose of this NER model. 

In [19]:
ORG, PER, LOC, MISC = [], [], [], []

for row in df['ner_output_unique']:
        if row != "[]":
            for entity in row:
                for key, value in entity.items():
                    if key == 'ORG':
                        ORG.append(value)
                    elif key == 'PER':
                        PER.append(value)
                    elif key == 'LOC':
                        LOC.append(value)
                    elif key == 'MISC':
                        MISC.append(value)

In [20]:
import pandas as pd
ORG_Series = pd.Series(ORG)
ORG_Series.value_counts().tail(30)

PayPal                   1
NorthFace                1
HenrySchein              1
FocusontheFamily         1
Greenpeace               1
BombasCharitableSocks    1
BassPro                  1
MorganChase              1
GoldmanSachs             1
HumanRightsCampaign      1
InandOut                 1
Pizza                    1
Ladys                    1
johnson                  1
WomankindWorldwide       1
GeneralMotors            1
VA                       1
Ikea                     1
WarbyParker              1
fila                     1
TechInnovate             1
Lion                     1
HomeDepot                1
EcoGoods                 1
UPS                      1
WorldLifeAssociation     1
Elf                      1
DairyQueen               1
Axiology                 1
BLQK                     1
Name: count, dtype: int64

In [21]:
LOC_Series = pd.Series(LOC)
LOC_Series.value_counts().head(10)

Palestine        4
Israel           2
America          2
Florida          2
US               2
NorthAmerica     1
NC               1
AmericanSouth    1
NorCal           1
Ukraine          1
Name: count, dtype: int64

In [22]:
MISC_Series = pd.Series(MISC)
MISC_Series.value_counts().head(10)

LGBTQ               86
Christian           31
Pride               13
PrideMonth          12
BlackLivesMatter     8
Black                5
Christianity         4
American             3
RacialJustice        3
SecondAmendment      2
Name: count, dtype: int64

In [23]:
PER_Series = pd.Series(PER)
PER_Series.value_counts().head(10)

Ben              4
Trump            3
DylanMulvaney    2
Rihanna          2
DonaldTrump      2
God              2
Tom              1
Hodgetwins       1
Kimkardashian    1
Jesus            1
Name: count, dtype: int64

In [24]:
link = 'labeled_answers.xlsx'
labeled_answers = pd.ExcelFile(link).parse('general_labeled')
labeled_answers.columns = labeled_answers.iloc[0]
labeled_answers = labeled_answers[1:].reset_index(drop=True)

In [25]:
df

,recognition_value,rv_label,ner_output,ner_output_unique,prediction_binary
0,"No, I cannot think of a firm that explicitly e...",Admit not knowing / remembering,[],[],1
1,McDonalds publicly supports families with sick...,Concrete example,"[{'ORG': 'McDonalds'}, {'ORG': 'RonaldMcDonald...","[{'ORG': 'McDonalds'}, {'ORG': 'RonaldMcDonald...",1
2,I cannot think of one but there are firms that...,Admit not knowing / remembering,[{'MISC': 'LGBTQ'}],[{'MISC': 'LGBTQ'}],0
3,Sackcloth makes blankets and works to be clima...,Concrete example,[],[],0
4,Bud Light and Dylan Mulvaney but they found ou...,Concrete example,"[{'ORG': 'BudLight'}, {'PER': 'DylanMulvaney'}...","[{'ORG': 'BudLight'}, {'PER': 'DylanMulvaney'}...",1
...,...,...,...,...,...
795,Victorias Secret supports LGBTQ,Concrete example,"[{'ORG': 'VictoriasSecret'}, {'MISC': 'LGBTQ'}]","[{'ORG': 'VictoriasSecret'}, {'MISC': 'LGBTQ'}]",1
796,Companies often align themselves with values r...,Generic example,[],[],0
797,The company BLQK puts 25% of their profit back...,Concrete example,[{'ORG': 'BLQK'}],[{'ORG': 'BLQK'}],1
798,Black Lives Matter supports racial inequities.,Nonsense,[{'MISC': 'BlackLivesMatter'}],[{'MISC': 'BlackLivesMatter'}],0


In [26]:
labeled_answers['ner_output_unique'] = df['ner_output_unique']

In [28]:
labeled_answers['party'].value_counts()

party
Independent      299
Democrat         266
Republican       219
No preference      8
Other              8
Name: count, dtype: int64

In [ ]:
def extract_orgs(ner_list):
    return [ent['ORG'] for ent in ner_list if 'ORG' in ent]
labeled_answers['orgs'] = labeled_answers['ner_output_unique'].apply(extract_orgs)

Grouping by Political Affiliations

In [ ]:
labeled_answers_exploded = labeled_answers[['party','orgs']].explode('orgs')

labeled_answers_exploded = labeled_answers_exploded.dropna(subset=['orgs'])

counts = (
    labeled_answers_exploded
    .groupby(['party','orgs'])
    .size()
    .reset_index(name='count')
)

counts_sorted = counts.sort_values(['party','count'], ascending=[True, False])

orgs_by_party = {
    party: grp.reset_index(drop=True)
    for party, grp in counts_sorted.groupby('party')
}

orgs_by_party['Republican']

,party,orgs,count
0,Republican,ChickfilA,18
1,Republican,Patagonia,8
2,Republican,BudLight,7
3,Republican,HobbyLobby,6
4,Republican,Target,6
5,Republican,NRA,5
6,Republican,Nike,5
7,Republican,Amazon,3
8,Republican,Dicks Sporting Goods,2
9,Republican,Disney,2


In [31]:
orgs_by_party['Democrat'].head(20)

,party,orgs,count
0,Democrat,Patagonia,14
1,Democrat,BenJerrys,12
2,Democrat,ChickfilA,9
3,Democrat,Starbucks,9
4,Democrat,Target,9
5,Democrat,Microsoft,6
6,Democrat,Nike,6
7,Democrat,Walmart,6
8,Democrat,HobbyLobby,5
9,Democrat,NAACP,4
